## 4. Tests of Normality

(1) Using the analogy principle, construct an estimator for the first k moments of the distribution of x. Use this to define a k-vector of moment restrictions $ g_{N}(\mu, \sigma) $ satisfying $ E g_{N}(\mu, \sigma) = 0  $ under the null hypothesis of normality.

In [19]:
import numpy as np
from math import factorial

def moment_conditions(mu, sigma, k, data):
    moments = [] # list to store vector of moments
    for m in range(1, k + 1):
        if m == 1:  # For m = 1, take the mean of the data:
            moments.append(np.mean(data) - 0)
        elif m % 2 == 1: 
        # For subsequent odd moments:
            moments.append(np.mean((data - mu) ** m) - 0)
        # For all even moments:
        else: 
            moments.append(np.mean((data - mu) ** m) - sigma ** m * factorial(m - 1))
    return np.array(moments)

In [20]:
# set seed for reproducibility
np.random.seed(1)

data = np.random.normal(0, 1, 10000000)  # create data from a normal distribution
k = 3  # Number of moment conditions
mu = 0  # true mean (for null hypothesis of normal dist)
sigma = 1  # true standard deviation (for null hypothesis of normal dist)

# Calculate moment conditions
moment_restriction_vector = moment_conditions(mu, sigma, k, data)

# Print moment restriction vector with labels
print("Moment Restriction Vector:")
for i, moment in enumerate(moment_restriction_vector):
    print(f"{i+1} moment:", moment)

Moment Restriction Vector:
1 moment: -8.572282554014973e-05
2 moment: -0.0004346238923502721
3 moment: -0.0006335370433218054


(2) What is the covariance matrix of the sample moment restrictions (again under the null)? I.e., what can be said about 
$ E g_{j}(\mu, \sigma) g_{j}(\mu, \sigma)⊤ $ - $ E g_{j}(\mu, \sigma) E g_{j}(\mu, \sigma)⊤ $?

The covariance matrix of the sample moment restrictions is a matrix of zeros. This is because the sample moment restrictions under the null hypothesis are expected to be constants. This means each moment restriction is independent of each other under the null hypothesis.

(3) Using your answers to the previous two questions, suggest a GMM-based test of the hypothesis of normality, taking k > 2.

(4) Implement the test you’ve devised using python. You may want to use scipy.stats.distributions.chi2.cdf and scipy.optimize.minimize.

(5) What can be said about the optimal choice of k?

(6) Compare the GMM estimates of (μ, σ) to the maximum likelihood estimates of these parameters. Do they differ? Why?